In [2]:
import yaml

def read_yaml_to_dict(file_path):
    try:
        with open(file_path, 'r') as file:
            yaml_data = yaml.safe_load(file)
        return yaml_data
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {file_path}")
        return None
    except yaml.YAMLError as e:
        print(f"YAML 파싱 오류: {e}")
        return None

In [3]:
# 사용 예시
file_path = 'config.yaml'
result = read_yaml_to_dict(file_path)

if result:
    print("YAML 내용:")
    print(result)

YAML 내용:
{'defaults': [{'override hydra/launcher': 'submitit_local'}], 'task': 'dog-run', 'obs': 'state', 'checkpoint': '???', 'eval_episodes': 10, 'eval_freq': 50000, 'steps': 10000000, 'batch_size': 256, 'reward_coef': 0.1, 'value_coef': 0.1, 'consistency_coef': 20, 'rho': 0.5, 'lr': '3e-4', 'enc_lr_scale': 0.3, 'grad_clip_norm': 20, 'tau': 0.01, 'discount_denom': 5, 'discount_min': 0.95, 'discount_max': 0.995, 'buffer_size': 1000000, 'exp_name': 'default', 'data_dir': '???', 'mpc': True, 'iterations': 6, 'num_samples': 512, 'num_elites': 64, 'num_pi_trajs': 24, 'horizon': 3, 'min_std': 0.05, 'max_std': 2, 'temperature': 0.5, 'log_std_min': -10, 'log_std_max': 2, 'entropy_coef': '1e-4', 'num_bins': 101, 'vmin': -10, 'vmax': 10, 'model_size': '???', 'num_enc_layers': 2, 'enc_dim': 256, 'num_channels': 32, 'mlp_dim': 512, 'latent_dim': 512, 'task_dim': 96, 'num_q': 5, 'dropout': 0.01, 'simnorm_dim': 8, 'wandb_project': 'dialectic-mujoco', 'wandb_entity': 'calmlab', 'wandb_silent': Fals

In [4]:
import torch

torch.cuda.is_available()

True

In [5]:
cfg = result
cfg['steps']

10000000

In [6]:
from omegaconf import OmegaConf
cfg = OmegaConf.load('config.yaml')

In [7]:
cfg.task

'dog-run'

In [8]:
domain, task = cfg.task.replace('-', '_').split('_', 1)
print (domain, task)

dog run


In [9]:
cfg.steps

10000000

In [45]:
from common.seed import set_seed
set_seed(cfg.seed)
cfg.task = 'walker-walk'

In [46]:
from pathlib import Path
work_dir = Path('/home/pipaek/project/tdmpc2') / 'logs' / cfg.task / str(cfg.seed) / cfg.exp_name

cfg.work_dir = work_dir
work_dir

PosixPath('/home/pipaek/project/tdmpc2/logs/walker-walk/1/default')

In [47]:
MODEL_SIZE = { # parameters (M)
	1:   {'enc_dim': 256,
		  'mlp_dim': 384,
		  'latent_dim': 128,
		  'num_enc_layers': 2,
		  'num_q': 2},
	5:   {'enc_dim': 256,
		  'mlp_dim': 512,
		  'latent_dim': 512,
		  'num_enc_layers': 2},
	19:  {'enc_dim': 1024,
		  'mlp_dim': 1024,
		  'latent_dim': 768,
		  'num_enc_layers': 3},
	48:  {'enc_dim': 1792,
		  'mlp_dim': 1792,
		  'latent_dim': 768,
		  'num_enc_layers': 4},
	317: {'enc_dim': 4096,
		  'mlp_dim': 4096,
		  'latent_dim': 1376,
		  'num_enc_layers': 5,
		  'num_q': 8},
}

In [48]:
TASK_SET = {
	'mt30': [
		# 19 original dmcontrol tasks
		'walker-stand', 'walker-walk', 'walker-run', 'cheetah-run', 'reacher-easy',
	    'reacher-hard', 'acrobot-swingup', 'pendulum-swingup', 'cartpole-balance', 'cartpole-balance-sparse',
		'cartpole-swingup', 'cartpole-swingup-sparse', 'cup-catch', 'finger-spin', 'finger-turn-easy',
		'finger-turn-hard', 'fish-swim', 'hopper-stand', 'hopper-hop',
		# 11 custom dmcontrol tasks
		'walker-walk-backwards', 'walker-run-backwards', 'cheetah-run-backwards', 'cheetah-run-front', 'cheetah-run-back',
		'cheetah-jump', 'hopper-hop-backwards', 'reacher-three-easy', 'reacher-three-hard', 'cup-spin',
		'pendulum-spin',
	],
	'mt80': [
		# 19 original dmcontrol tasks
		'walker-stand', 'walker-walk', 'walker-run', 'cheetah-run', 'reacher-easy',
	    'reacher-hard', 'acrobot-swingup', 'pendulum-swingup', 'cartpole-balance', 'cartpole-balance-sparse',
		'cartpole-swingup', 'cartpole-swingup-sparse', 'cup-catch', 'finger-spin', 'finger-turn-easy',
		'finger-turn-hard', 'fish-swim', 'hopper-stand', 'hopper-hop',
		# 11 custom dmcontrol tasks
		'walker-walk-backwards', 'walker-run-backwards', 'cheetah-run-backwards', 'cheetah-run-front', 'cheetah-run-back',
		'cheetah-jump', 'hopper-hop-backwards', 'reacher-three-easy', 'reacher-three-hard', 'cup-spin',
		'pendulum-spin',
		# meta-world mt50
		'mw-assembly', 'mw-basketball', 'mw-button-press-topdown', 'mw-button-press-topdown-wall', 'mw-button-press',
		'mw-button-press-wall', 'mw-coffee-button', 'mw-coffee-pull', 'mw-coffee-push', 'mw-dial-turn',
		'mw-disassemble', 'mw-door-open', 'mw-door-close', 'mw-drawer-close', 'mw-drawer-open',
		'mw-faucet-open', 'mw-faucet-close', 'mw-hammer', 'mw-handle-press-side', 'mw-handle-press',
		'mw-handle-pull-side', 'mw-handle-pull', 'mw-lever-pull', 'mw-peg-insert-side', 'mw-peg-unplug-side',
		'mw-pick-out-of-hole', 'mw-pick-place', 'mw-pick-place-wall', 'mw-plate-slide', 'mw-plate-slide-side',
		'mw-plate-slide-back', 'mw-plate-slide-back-side', 'mw-push-back', 'mw-push', 'mw-push-wall',
		'mw-reach', 'mw-reach-wall', 'mw-shelf-place', 'mw-soccer', 'mw-stick-push',
		'mw-stick-pull', 'mw-sweep-into', 'mw-sweep', 'mw-window-open', 'mw-window-close',
		'mw-bin-picking', 'mw-box-close', 'mw-door-lock', 'mw-door-unlock', 'mw-hand-insert',
	],
}

In [49]:
cfg.task_title = cfg.task.replace("-", " ").title()
cfg.bin_size = (cfg.vmax - cfg.vmin) / (cfg.num_bins-1) # Bin size for discrete regression

# Model size
if cfg.get('model_size', None) is not None:
    assert cfg.model_size in MODEL_SIZE.keys(), \
        f'Invalid model size {cfg.model_size}. Must be one of {list(MODEL_SIZE.keys())}'
    for k, v in MODEL_SIZE[cfg.model_size].items():
        cfg[k] = v
    if cfg.task == 'mt30' and cfg.model_size == 19:
        cfg.latent_dim = 512 # This checkpoint is slightly smaller

# Multi-task
cfg.multitask = cfg.task in TASK_SET.keys()
if cfg.multitask:
    cfg.task_title = cfg.task.upper()
    # Account for slight inconsistency in task_dim for the mt30 experiments
    cfg.task_dim = 96 if cfg.task == 'mt80' or cfg.model_size in {1, 317} else 64
else:
    cfg.task_dim = 0
cfg.tasks = TASK_SET.get(cfg.task, [cfg.task])

In [50]:
cfg.task_title

'Walker Walk'

In [51]:
cfg.bin_size

0.2

In [52]:
cfg.model_size   # 아직 안정해졌나봄봄

MissingMandatoryValue: Missing mandatory value: model_size
    full_key: model_size
    object_type=dict

In [53]:
cfg.multitask

False

In [54]:
cfg.task_dim = 0

In [55]:
cfg.tasks

['walker-walk']

In [56]:
from envs import make_env
env=make_env(cfg)

In [57]:
cfg.get('obs', 'state')

'state'

In [23]:
cfg.obs_shape

{'state': [24]}

In [30]:
obs = env.reset()
randact = env.rand_act()
env.step(action)
obs

tensor([ 0.5650, -0.8251,  0.1182, -0.9930,  0.9702, -0.2425,  0.9930, -0.1177,
        -0.4668, -0.8844,  0.9997,  0.0263,  0.8432, -0.5376,  1.3000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000])

In [31]:
randact = env.rand_act()
randact

tensor([-0.1887, -0.2055, -0.8213,  0.1226, -0.4204,  0.5166])

In [36]:
act_l = randact[action_filter_l]
act_l

tensor([-0.1887, -0.2055, -0.8213])

In [40]:
act_r = randact[action_filter_r]
act_r

tensor([ 0.1226, -0.4204,  0.5166])

In [41]:
act = torch.concat([act_l, act_r], dim=0)
act

tensor([-0.1887, -0.2055, -0.8213,  0.1226, -0.4204,  0.5166])

In [26]:
obs = obs.unsqueeze(0).cpu()
obs

tensor([[ 0.8671, -0.4981, -0.1099, -0.9939,  0.5924,  0.8056,  0.3179,  0.9481,
          0.8871, -0.4615, -0.3201,  0.9474, -0.7302,  0.6832,  1.3000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]])

In [27]:
action = torch.full_like(env.rand_act(), float('nan'))
action

tensor([nan, nan, nan, nan, nan, nan])

In [28]:
reward = torch.tensor(float('nan'))
reward

tensor(nan)

In [29]:
from tensordict.tensordict import TensorDict
td = TensorDict(dict(
			obs=obs,
			action=action.unsqueeze(0),
			reward=reward.unsqueeze(0),
		), batch_size=(1,))
td

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([1, 6]), device=cpu, dtype=torch.float32, is_shared=False),
        obs: Tensor(shape=torch.Size([1, 24]), device=cpu, dtype=torch.float32, is_shared=False),
        reward: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.float32, is_shared=False)},
    batch_size=torch.Size([1]),
    device=None,
    is_shared=False)

In [32]:
ACTION_DESCRIPTIONS = [
    {'index': 0, 'object': 'right_hip', 'min': -1, 'max': 1, 'object_type': 'motor', 'action_type': 'continuous', 'left_actor': True, 'right_actor': False},
    {'index': 1, 'object': 'right_knee', 'min': -1, 'max': 1, 'object_type': 'motor', 'action_type': 'continuous', 'left_actor': True, 'right_actor': False},
    {'index': 2, 'object': 'right_ankle', 'min': -1, 'max': 1, 'object_type': 'motor', 'action_type': 'continuous', 'left_actor': True, 'right_actor': False},
    {'index': 3, 'object': 'left_hip', 'min': -1, 'max': 1, 'object_type': 'motor', 'action_type': 'continuous', 'left_actor': False, 'right_actor': True},
    {'index': 4, 'object': 'left_knee', 'min': -1, 'max': 1, 'object_type': 'motor', 'action_type': 'continuous', 'left_actor': False, 'right_actor': True},
    {'index': 5, 'object': 'left_ankle', 'min': -1, 'max': 1, 'object_type': 'motor', 'action_type': 'continuous', 'left_actor': False, 'right_actor': True}]
OBSERVATION_DESCRIPTIONS = [
    {'index': 0, 'object': 'torso', 'object_type': 'orientation', 'axis': 'xx', 'left_actor': True, 'right_actor': True},
    {'index': 1, 'object': 'torso', 'object_type': 'orientation', 'axis': 'xz', 'left_actor': True, 'right_actor': True},
    {'index': 2, 'object': 'right_thigh', 'object_type': 'orientation', 'axis': 'xx', 'left_actor': True, 'right_actor': False},
    {'index': 3, 'object': 'right_thigh', 'object_type': 'orientation', 'axis': 'xz', 'left_actor': True, 'right_actor': False},
    {'index': 4, 'object': 'right_leg', 'object_type': 'orientation', 'axis': 'xx', 'left_actor': True, 'right_actor': False},
    {'index': 5, 'object': 'right_leg', 'object_type': 'orientation', 'axis': 'xz', 'left_actor': True, 'right_actor': False},
    {'index': 6, 'object': 'right_foot', 'object_type': 'orientation', 'axis': 'xx', 'left_actor': True, 'right_actor': False},
    {'index': 7, 'object': 'right_foot', 'object_type': 'orientation', 'axis': 'xz', 'left_actor': True, 'right_actor': False},
    {'index': 8, 'object': 'left_thigh', 'object_type': 'orientation', 'axis': 'xx', 'left_actor': False, 'right_actor': True},
    {'index': 9, 'object': 'left_thigh', 'object_type': 'orientation', 'axis': 'xz', 'left_actor': False, 'right_actor': True},
    {'index': 10, 'object': 'left_leg', 'object_type': 'orientation', 'axis': 'xx', 'left_actor': False, 'right_actor': True},
    {'index': 11, 'object': 'left_leg', 'object_type': 'orientation', 'axis': 'xz', 'left_actor': False, 'right_actor': True},
    {'index': 12, 'object': 'left_foot', 'object_type': 'orientation', 'axis': 'xx', 'left_actor': False, 'right_actor': True},
    {'index': 13, 'object': 'left_foot', 'object_type': 'orientation', 'axis': 'xz', 'left_actor': False, 'right_actor': True},
    {'index': 14, 'object': 'torso', 'object_type': 'height', 'left_actor': True, 'right_actor': True},
    {'index': 15, 'object': 'rootz', 'object_type': 'velocity', 'left_actor': True, 'right_actor': True},
    {'index': 16, 'object': 'rootx', 'object_type': 'velocity', 'left_actor': True, 'right_actor': True},
    {'index': 17, 'object': 'rooty', 'object_type': 'velocity', 'left_actor': True, 'right_actor': True},
    {'index': 18, 'object': 'right_hip', 'object_type': 'velocity', 'left_actor': True, 'right_actor': False},
    {'index': 19, 'object': 'right_knee', 'object_type': 'velocity', 'left_actor': True, 'right_actor': False},
    {'index': 20, 'object': 'right_ankle', 'object_type': 'velocity', 'left_actor': True, 'right_actor': False},
    {'index': 21, 'object': 'left_hip', 'object_type': 'velocity', 'left_actor': False, 'right_actor': True},
    {'index': 22, 'object': 'left_knee', 'object_type': 'velocity', 'left_actor': False, 'right_actor': True},
    {'index': 23, 'object': 'left_ankle', 'object_type': 'velocity', 'left_actor': False, 'right_actor': True},
]

In [33]:
action_filter_l = [a['left_actor'] for a in ACTION_DESCRIPTIONS]
action_filter_l

[True, True, True, False, False, False]

In [39]:
action_filter_r = [a['right_actor'] for a in ACTION_DESCRIPTIONS]
action_filter_r

[False, False, False, True, True, True]

In [34]:
obj_filter_l = [a['left_actor'] for a in OBSERVATION_DESCRIPTIONS]
obj_filter_l

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False]

In [35]:
obs_l = obs[obj_filter_l]
obs_l

tensor([ 0.5650, -0.8251,  0.1182, -0.9930,  0.9702, -0.2425,  0.9930, -0.1177,
         1.3000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000])

In [51]:
obs = obs.to('cuda', non_blocking=True)#.unsqueeze(0)
obs

tensor([ 0.5650, -0.8251,  0.1182, -0.9930,  0.9702, -0.2425,  0.9930, -0.1177,
        -0.4668, -0.8844,  0.9997,  0.0263,  0.8432, -0.5376,  1.3000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
       device='cuda:0')

In [52]:
last_action_l = torch.zeros(3, device='cuda')
last_action_l

tensor([0., 0., 0.], device='cuda:0')

In [53]:
torch.concat([obs, last_action_l], dim=0)

tensor([ 0.5650, -0.8251,  0.1182, -0.9930,  0.9702, -0.2425,  0.9930, -0.1177,
        -0.4668, -0.8844,  0.9997,  0.0263,  0.8432, -0.5376,  1.3000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000], device='cuda:0')

In [37]:
# from envs.dmcontrol import make_env as make_dm_control_env

In [40]:
# from envs.myosuite import make_env as make_myosuite_env

In [41]:
# env = make_myosuite_env(cfg)

ValueError: ('Unknown task:', 'walker-walk')

In [43]:
from tdmpc2 import TDMPC2
agent=TDMPC2(cfg)

/home/pipaek/project/tdmpc2/tdmpc2/common/layers.py:15: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.combine_state_for_ensemble` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.func.stack_module_state` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  fn, params, _ = combine_state_for_ensemble(modules)


In [44]:
from common.buffer import Buffer
buffer=Buffer(cfg)

/home/pipaek/miniconda3/envs/tdmpc2/lib/python3.9/site-packages/torchrl/data/replay_buffers/samplers.py:37: UserWarning: Failed to import torchrl C++ binaries. Some modules (eg, prioritized replay buffers) may not work with your installation. If you installed TorchRL from PyPI, please report the bug on TorchRL github. If you installed TorchRL locally and/or in development mode, check that you have all the required compiling packages.
  warnings.warn(EXTENSION_WARNING)


In [45]:
from common.logger import Logger
logger=Logger(cfg)

------------------------------------
  Task:           Walker Walk
  Steps:          10,000,000
  Observations:   [24]
  Actions:        6
  Experiment:     default
------------------------------------


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: uhmppi1 (calmlab). Use `wandb login --relogin` to force relogin


Logs will be synced with wandb.


In [46]:
from trainer.online_trainer import OnlineTrainer
trainer = OnlineTrainer(
		cfg=cfg,
		env=env,
		agent=agent,
		buffer=buffer,
		logger=logger,
	)


Architecture: WorldModel(
  (_encoder): ModuleDict(
    (state): Sequential(
      (0): NormedLinear(in_features=24, out_features=256, bias=True, act=Mish)
      (1): NormedLinear(in_features=256, out_features=512, bias=True, act=SimNorm)
    )
  )
  (_dynamics): Sequential(
    (0): NormedLinear(in_features=518, out_features=512, bias=True, act=Mish)
    (1): NormedLinear(in_features=512, out_features=512, bias=True, act=Mish)
    (2): NormedLinear(in_features=512, out_features=512, bias=True, act=SimNorm)
  )
  (_reward): Sequential(
    (0): NormedLinear(in_features=518, out_features=512, bias=True, act=Mish)
    (1): NormedLinear(in_features=512, out_features=512, bias=True, act=Mish)
    (2): Linear(in_features=512, out_features=101, bias=True)
  )
  (_pi): Sequential(
    (0): NormedLinear(in_features=512, out_features=512, bias=True, act=Mish)
    (1): NormedLinear(in_features=512, out_features=512, bias=True, act=Mish)
    (2): Linear(in_features=512, out_features=12, bias=True

In [47]:
agent.device = torch.device('cuda')
agent.device

device(type='cuda')

In [48]:
from common.world_model import WorldModel
agent.model = WorldModel(cfg).to(agent.device)

/home/pipaek/project/tdmpc2/tdmpc2/common/layers.py:15: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.combine_state_for_ensemble` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.func.stack_module_state` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  fn, params, _ = combine_state_for_ensemble(modules)


In [49]:
for k in cfg.obs_shape.keys():
    print(k, cfg.obs_shape[k])

state [24]


In [ ]:
cfg.action_dim = env.action_space.shape[0]
cfg.episode_length = env.max_episode_steps
cfg.seed_steps = max(1000, 5*cfg.episode_length)

In [50]:
env.action_space.shape[0]

6

In [51]:
env.max_episode_steps

500

In [52]:
cfg.episode_length

500

In [54]:
env.observation_space

Box([-inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf], [inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf], (24,), float32)

In [55]:
env.observation_space.spaces

AttributeError: 'Box' object has no attribute 'spaces'

In [56]:
env.observation_spec()

AttributeError: 'TimeStepToGymWrapper' object has no attribute 'observation_spec'

In [57]:
domain, task = cfg.task.replace('-', '_').split('_', 1)

In [58]:
domain

'walker'

In [59]:
task

'walk'

In [60]:
domain = dict(cup='ball_in_cup', pointmass='point_mass').get(domain, domain)
domain

'walker'

In [61]:
from dm_control import suite
env = suite.load(domain,
					 task,
					 task_kwargs={'random': cfg.seed},
					 visualize_reward=False)


In [62]:
env.observation_spec()

OrderedDict([('orientations',
              Array(shape=(14,), dtype=dtype('float64'), name='orientations')),
             ('height',
              Array(shape=(), dtype=dtype('float64'), name='height')),
             ('velocity',
              Array(shape=(9,), dtype=dtype('float64'), name='velocity'))])

In [63]:
env.action_spec()

BoundedArray(shape=(6,), dtype=dtype('float64'), name=None, minimum=[-1. -1. -1. -1. -1. -1.], maximum=[1. 1. 1. 1. 1. 1.])

In [22]:
for k in cfg.obs_shape.keys():
    print(k, cfg.obs_shape[k])

state [24]


In [23]:
cfg.obs_shape[k][0] + cfg.task_dim

24

In [24]:
cfg.task_dim

0

In [25]:
cfg.num_enc_layers

2

In [26]:
cfg.enc_dim

256

In [27]:
max(cfg.num_enc_layers-1, 1)*[cfg.enc_dim]

[256]

In [28]:
cfg.latent_dim

512

In [29]:
cfg.simnorm_dim

8

In [30]:
cfg.action_dim

6

# REINFORCE

In [71]:
from envs import make_env
env=make_env(cfg)

In [76]:
cfg.task

'walker-walk'

In [72]:
obs = env.reset()


In [73]:
randact = env.rand_act()
randact = torch.tensor([0, 0, 0, 0, 0, 0], device='cuda')
randact


tensor([0, 0, 0, 0, 0, 0], device='cuda:0')

In [74]:
obs, reward, done, info = env.step(action)

PhysicsError: Physics state is invalid. Warning(s) raised: mjWARN_BADCTRL

In [75]:
env.action_space

Box([-1. -1. -1. -1. -1. -1.], [1. 1. 1. 1. 1. 1.], (6,), float32)

In [62]:
env.action_space.sample()

array([-0.4854661 ,  0.6615692 ,  0.45696422,  0.32358095, -0.06977106,
       -0.9945168 ], dtype=float32)

In [ ]:
def to_td(obs, action=None, reward=None):
		"""Creates a TensorDict for a new episode."""
		if isinstance(obs, dict):
			obs = TensorDict(obs, batch_size=(), device='cpu')
		else:
			obs = obs.unsqueeze(0).cpu()
		if action is None:
			action = torch.full_like(env.rand_act(), float('nan'))
		if reward is None:
			reward = torch.tensor(float('nan'))
		td = TensorDict(dict(
			obs=obs,
			action=action.unsqueeze(0),
			reward=reward.unsqueeze(0),
		), batch_size=(1,))
		return td